In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
raw_data_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'raw')
csv_files = [f for f in os.listdir(raw_data_dir) if f.endswith('.csv')]

print(f"\nFound {len(csv_files)} CSV files:")
for f_name in csv_files:
        print(f"- {f_name}")


Found 31 CSV files:
- android_metadata.csv
- ascents.csv
- attempts.csv
- beta_links.csv
- bids.csv
- circuits.csv
- circuits_climbs.csv
- climbs.csv
- climb_cache_fields.csv
- climb_random_positions.csv
- climb_stats.csv
- difficulty_grades.csv
- holes.csv
- kits.csv
- layouts.csv
- leds.csv
- placements.csv
- placement_roles.csv
- products.csv
- products_angles.csv
- product_sizes.csv
- product_sizes_layouts_sets.csv
- sets.csv
- shared_syncs.csv
- sqlite_stat1.csv
- tags.csv
- users.csv
- user_permissions.csv
- user_syncs.csv
- walls.csv
- walls_sets.csv


In [3]:
#Empty csv: android_metadata, ascents, bids, circuits, circuits_climb, climb_random_positions, tags, users, user_permissions, user_syncs, walls, walls_sets

#Useless csv: beta_links, sets, shared_syncs, sqlite_stat1

#No idea csv: attempts, kits, layouts, leds, products, products_angles, product_sizes, product_sizes_layouts_sets

#Useful csv: climbs, climb_cache_fields, climb_stats, difficulty_grades, holes, placements, placement_roles

'c:\\Users\\mdaman\\CodeProjects\\ai_Kilter_Routes\\data\\raw'

In [5]:
climbs_csv_path = os.path.join(raw_data_dir, 'climbs.csv')
climbStats_csv_path = os.path.join(raw_data_dir, 'climb_stats.csv')
climbCache_csv_path = os.path.join(raw_data_dir, 'climb_cache_fields.csv')


df_climbs = pd.read_csv(climbs_csv_path)
df_climbStats = pd.read_csv(climbStats_csv_path)
df_climbCache = pd.read_csv(climbCache_csv_path)

In [6]:
print(df_climbs.columns)
#uuid = climb identifier
#layout_id = board identifier
#hsm = Hold Set Manufacturer????
#edge_... = ????
#frames_count = only want 1
#frames = the holds on the frame

#drop the following columns: setter_username, name, description, hsm, edge_...., created_at
df_climbs = df_climbs.drop(['setter_username', 'description', 'hsm', 
                            'edge_left', 'edge_right', 'edge_bottom','edge_top','created_at'], axis=1)

Index(['uuid', 'layout_id', 'setter_id', 'setter_username', 'name',
       'description', 'hsm', 'edge_left', 'edge_right', 'edge_bottom',
       'edge_top', 'angle', 'frames_count', 'frames_pace', 'frames',
       'is_draft', 'is_listed', 'created_at', 'is_nomatch'],
      dtype='object')


In [7]:
df_climbStats = df_climbStats.rename(columns={'climb_uuid': 'uuid'})
df_climbCache = df_climbCache.rename(columns={'climb_uuid': 'uuid'})

In [8]:
merged_df = pd.merge(df_climbs, df_climbStats, on='uuid')
merged_df = pd.merge(merged_df, df_climbCache, on='uuid')

In [9]:
merged_df_final = merged_df[merged_df["is_listed"] == 1]
merged_df_final = merged_df_final[merged_df_final["is_draft"] == 0]
merged_df_final = merged_df_final[merged_df_final["frames_count"] == 1]
merged_df_final = merged_df_final[merged_df_final["layout_id"] == 1]
merged_df_final = merged_df_final[merged_df_final["frames_pace"] == 0]
merged_df_final = merged_df_final[merged_df_final["ascensionist_count_x"] > 9]
merged_df_final = merged_df_final[merged_df_final["ascensionist_count_y"] > 9]

merged_df_final = merged_df_final.drop(['fa_username', 'fa_at', 'angle_x'], axis=1)
merged_df_final = merged_df_final.drop(['ascensionist_count_y', 'display_difficulty_y', 'quality_average_y', 'benchmark_difficulty'], axis=1)

In [10]:
#merged_df_final[(merged_df_final['difficulty_average'] - merged_df_final['display_difficulty_x']).abs() > 0.3]
merged_df_final = merged_df_final.drop(['display_difficulty_x'], axis=1)
merged_df_final = merged_df_final.rename(columns={'angle_y': 'angle', 'difficulty_average': 'difficulty', 
                                                  'quality_average_x': 'quality', 'ascensionist_count_x': 'ascensionist_count'})


In [11]:
print(merged_df_final['angle'].isna().any())
print(merged_df_final['difficulty'].isna().any())

False
False


In [12]:
for column_name in merged_df_final.columns:
    print(f"{column_name}:", merged_df_final[column_name].isna().sum())

uuid: 0
layout_id: 0
setter_id: 0
name: 0
frames_count: 0
frames_pace: 0
frames: 0
is_draft: 0
is_listed: 0
is_nomatch: 0
angle: 0
ascensionist_count: 0
difficulty: 0
quality: 0


In [14]:
merged_df_final[merged_df_final['name'] == 'Master Chief']

,uuid,layout_id,setter_id,name,frames_count,frames_pace,frames,is_draft,is_listed,is_nomatch,angle,ascensionist_count,difficulty,quality
19757,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,70,10,22.3000,2.80000
19758,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,65,20,22.9500,2.85000
19759,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,20,37,19.7297,2.86486
19760,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,55,43,20.9302,2.90698
19761,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,0,66,19.5909,2.59091
19762,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,60,226,22.8407,2.95575
19763,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,35,429,20.8904,2.95338
19764,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,30,450,19.9800,2.95778
19765,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,25,38,20.0526,2.78947
19766,28EFC798ECE24D55A15AC0FE23FA986B,1,27006,Master Chief,1,0,p1145r12p1163r12p1215r13p1236r13p1289r13p1324r...,0,1,0,50,15668,21.9756,2.98500
